In [1]:
# COLLEGAMENTO CON GOOGLE DRIVE
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
# LIBRERIE E CONFIGURAZIONE DI OPEN ROUTER

import pandas as pd
import json
import os
import requests
import time
from io import StringIO
import re
from datetime import datetime
from bs4 import BeautifulSoup

# Configura l'endpoint di OpenRouter
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
API_KEY = "key"
# Configura l'intestazione per l'API
headers = {
    "Authorization": f"Bearer {API_KEY}",
    #"Accept": "application/json",
    "Content-Type": "text/plain"
}


In [2]:
# FUNZIONI PER PARSING E SERIALIZZAZIONE DELLE TABELLE JSON

def parse_html_table_with_html5lib(html_table):
    #Parsa una tabella HTML in un DataFrame pandas gestendo automaticamente rowspan e colspan."""
    # Wrapping in StringIO
    html_data = StringIO(html_table)

    # Usa pandas per leggere la tabella HTML
    df = pd.read_html(html_data, flavor='html5lib')[0]

    #Riempire i NaN lungo le righe
    df = df.ffill(axis=0)

    # Riempire i NaN lungo le colonne (se necessario)
    df = df.ffill(axis=1)

    return df

def serialize_json_tables(json_file_path,filename):
    """Carica un file JSON con tabelle e le serializza in un elenco di DataFrame."""
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    all_tables = []  # Contenitore per tutte le tabelle
    all_contexts = [] # lista di liste dei contesti delle tabelle

    # Itera su ogni tabella nel JSON
    for key, table_info in data.items():
        html_table = table_info.get('table', '')
        caption = table_info.get('caption', '')
        footnotes = table_info.get('footnotes', [])
        references = table_info.get('references', [])
        all_contexts.append([caption,references,footnotes,filename])

        # Parsa la tabella HTML
        if html_table:
            df = parse_html_table_with_html5lib(html_table)
            all_tables.append(df)

    return all_tables,all_contexts


In [32]:

def send_request(payload):
    try:
        print("Invio richiesta al LLM...")
        response = requests.post(OPENROUTER_URL, headers=headers, json=payload)

        # Controllo del codice di stato della risposta
        if response.status_code == 200:
            try:
                json_response = response.json()

                # Gestione di errori specifici nel corpo della risposta
                if "error" in json_response:
                    error_code = json_response["error"].get("code")

                    if error_code == 429:
                        message=json_response["error"].get("message")
                        retry_after = json_response["error"].get("metadata", {}).get("retry_after", 5)
                        print(f"Errore 429: {message}. Riprovo tra {retry_after} secondi...")
                        time.sleep(retry_after)
                        return send_request(payload)

                    else:
                        print(f"Errore sconosciuto: {json_response['error']}")
                        dumps=json.dumps({"errore": json_response["error"].get("message")})
                        return dumps
                # Nessun errore: restituisci il JSON della risposta
                print("Risposta ricevuta correttamente.")
                return json_response

            except ValueError as e:
                # Errore nel parsing del JSON
                print(f"Errore di parsing JSON: {e}")
                dumps=json.dumps({"errore": "Errore di parsing JSON"})
                return dumps


        else:
            # Gestione di errori HTTP non 200
            print(f"Errore HTTP {response.status_code}: {response.text}")
            dumps=json.dumps({"errore": f"HTTP {response.status_code}"})
            return dumps

    except requests.RequestException as e:
        # Gestione di errori nella richiesta
        print(f"Errore di connessione: {e}")
        dumps=json.dumps({"errore": "Errore di connessione"})
        return dumps


# Estarzione del claim in json
def estrai_json(response):
    
    #Estrae una sezione di testo compresa tra due delimitatori.
    start_delimiter="```json"
    end_delimiter="```"

    try:
        # Costruisce il pattern con espressioni regolari
        pattern = rf"{re.escape(start_delimiter)}\s*(.*?)\s*{re.escape(end_delimiter)}"
        match = re.search(pattern, response, re.DOTALL)


        # Restituisce il risultato se trovato
        if match:
            return match.group(1).strip("\n")  # Rimuove spazi bianchi inutili
        else:
            dumps=json.dumps({"errore": "Nessuna corrispondenza nella risposta da LLM"})
            return dumps

    except Exception as e:
        print(f"Errore durante l'estrazione: {e}")
        dumps=json.dumps({"errore": "Nessuna corrispondenza nella risposta da LLM"})
        return dumps




###funzione semplice per salvare l'output in un file json
####serve per creare il file json dei claim

def salva_in_file(nome_file, testo, encoding='utf-8'):
    
    # Salva il testo in un file specificato json

    try:
        # Apri il file in modalità scrittura ('w') con l'encoding specificato
        with open(nome_file, 'w', encoding=encoding) as f:
            json.dump(testo, f, indent=4, ensure_ascii=False)
        print(f"Il testo è stato salvato con successo nel file: {nome_file}")
    except Exception as e:
        print(f"Si è verificato un errore: {e}")



## funzione per stampare json serializzato
### salva la risposta del LLM in formato json dopo averla serializzata in caso non lo sia

def salva_risposta_json(response_content):

    # Encoder personalizzato per tipi non serializzabili in JSON.
    def custom_encoder(obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # Converte datetime in stringa ISO 8601
        elif isinstance(obj, set):
            return list(obj)  # Converte set in lista
        elif hasattr(obj, "__dict__"):
            return obj.__dict__  # Serializza oggetti con attributi
        raise TypeError(f"Oggetto di tipo {type(obj).__name__} non serializzabile.")

    try:
        risposta_json=json.dumps(response_content, indent=4, ensure_ascii=False, default=custom_encoder)
        return risposta_json
        print(f"Risposta salvata con successo.")
    except Exception as e:
        print(f"Errore durante il salvataggio della risposta: {e}")

# Funzione per generare claims con LLM
def generate_claims_with_llm(tables, contexts):
    # Crea un prompt da inviare al modello Gemini
    claims = []
    print(len(tables))

    for index ,(table, context) in enumerate(zip(tables, contexts)):
        caption, references, footnotes,filename = context
          # Estrai il paperID dal filename
        match = re.match(r"(\d+\.\d+)", filename)
        if match:
          paperID = match.group(1)
        else:
          paperID = "unknown"  # Valore di default nel caso non ci sia un match
       # Controlla se footnotes e references sono non vuoti e formatta di conseguenza
        footnotes_str = '; '.join(footnotes) if footnotes else 'None'
        references_str = '; '.join(references) if references else 'None'
        # Creazione di un prompt per il modello

        # Payload per la richiesta
        payload = {
            "model": "google/gemini-2.0-flash-thinking-exp:free",
            "messages": [
        {
           "role": "system",
           "content": "Sei un assistente che restituisce esclusivamente risposte in formato JSON strutturato."
        },
        {
        "role": "user",
        "content": f"""
        Filename: {filename}
        Table: {table.to_string(index=False)}
        Caption: {caption}
        Footnotes: {'; '.join(footnotes_str)}
        References: {'; '.join(references_str)} """
        """
        Required Output: extract claims from the each tables: a claim is a sequence of
        of specifications, followed by Measure and Outcome.
        A specification is a pair name: "specification name","value" describing some details.
        Usually there's a claim for each outcome of each record of the table. The name of a specification
        can be refered to a column name or can be an integration between the table and
        the context: caption,references, footnotes.
        Measure indicates the method used for evaluation anche outcome is the value.

        An example of table with caption and paragraph and correct claims format

              Table and caption:                           Paragraph:
         TABLE IV: Partition characteristics        We adopted n = 500, i.e. the RL-graph of each dataset
        Dataset    # cutting edgesa              was partitioned into 500 subgraphs using METIS. The effects
        LUBM-8000    23,624,351    1.23
        LUBM-20480   61,518,672    1.21           of PMETIS and PI-UHC is given in Table IV, manifested as the
        SNIB-15000   58,823,356    1.52           number of cutting edges and replication factor a respectively.

        1{1Dataset, LUBM-80001, I RL-GRAPH partitions, 500 subgraphs I, 1# cutting edges, 23,624,3511, I replication factor alpha, 1.231}1
        1{1Dataset, LUBM-204801, I RL-GRAPH partitions, 500 subgraphs I, 1# cutting edges, 61,518,6721, I replication factor alpha, 1.211}1
        1{1Dataset, 5141B-150001, I RL-GRAPH partitions, 500 subgraphs I, 1# cutting edges, 58,823,3561, I replication factor alpha, 1.521}1

         after claim extraction, use the json structure example below to organize the claims:
         [
           {
            "0": {
              "specifications": {
                 "0": { "name":"Model Type", "value":"General LLM"},
                 "1": { "name":"Model Name", "value":"ChatGPT-3.5-turbo"},
                 "2": { "name":"Parameter Size", "value":"175B"},
                 "3": { "name":"Dataset", "value":"Spider dev"},
                 "4": { "name":"Difficulty Level", "value":"1"}
                               },
              "Measure": "Execution Match",
               "Outcome": "0.760"
               }
          },
          {
            "1": {
              "specifications": {
                 "0": { "name":"Model Type", "value":"General LLM"},
                 "1": {"name":"Model Name",  "value":"GPT-4"},
                 "2": {"name":"Parameter Size", "value":"200B"},
                 "3": { "name":"Dataset",  "value":"SQuAD dev"},
                 "4": { "name":"Difficulty Level",  "value":"2"}
                                },
             "Measure": "Execution Match",
             "Outcome": "0.840"
                }
            }
         ]

         Please responde only with the part related to the JSON structure, without the explanation
         and create a different json for every table in each filename and restart from 0 the count of claims
         for each table.
         """
         }
        ],

          "max_tokens": 8000  # Imposta un limite sul numero di token generati


        }
        print(filename) # debug
        response_content = send_request(payload)
        #if response_content
         # response_content=json.loads(response_content)
        json_serial= salva_risposta_json(response_content)
       # salva_in_file(nome_file=f"/content/drive/My Drive/Ingegneria-dei-dati/response_{index+1}.json", testo= json.loads(json_serial))  #debug;
        time.sleep(5)

        json_serial=json.loads(json_serial)

        try:
          #se il json è valido e contiene claim estrae il contenuto dei claim e li salva in file
         if "choices" in json_serial and isinstance(json_serial["choices"], list) and len(json_serial["choices"]) > 0:
            content = json_serial["choices"][0]["message"]["content"]
            json_string = estrai_json(content)  # Estrai il contenuto tra ```json\n e \n```
            # tolto try
            salva_in_file(nome_file=f"/content/drive/My Drive/ing_dati/{paperID}_{index+1}_claims.json", testo= json.loads(json_string))

         else:
            #nel caso sia il json di risposta lo salva ugualmente del file segnando errore
            salva_in_file(nome_file=f"/content/drive/My Drive/ing_dati/{paperID}_{index+1}_error.json", testo=json.loads(json_serial))

        except json.JSONDecodeError as e:
            print(f"Errore JSONDecodeError: {e}, skipping this table in file {filename}")
            print(f"The problematic json string is: {json_string}")  # Print the problematic string for debugging
            continue  # Skip to the next table
    return claims



In [33]:
# Lista per contenere tutte le tabelle da tutti i file JSON
all_tables = []
all_contexts = []
json_folder_path = '/content/drive/My Drive/ing_dati/json'

# Itera su tutti i file JSON nella cartella
for filename in os.listdir(json_folder_path):
    if filename.endswith('.json'):
        json_file_path = os.path.join(json_folder_path, filename)

        # Serializza le tabelle dal file JSON corrente
        serialized_tables,contexts = serialize_json_tables(json_file_path,filename)

        # Aggiungi le tabelle alla lista globale
        # extend serve per concatenare liste
        all_tables.extend(serialized_tables)
        all_contexts.extend(contexts)



with open("/content/drive/My Drive/ing_dati/tabelle.txt", 'w') as f:
    for index, (table, context) in enumerate(zip(all_tables, all_contexts)):
        # Crea la stringa da scrivere nel file
        output_str = f"Tabella {index} file {context[3]}:\n{table}\n\n"

        # Scrive la stringa nel file
        f.write(output_str)

# Genera claims a partire dalle tabelle
claims = generate_claims_with_llm(all_tables, all_contexts)
output_file="/content/drive/My Drive/Ingegneria-dei-dati/claims.txt"


31
2212.01552v1.json
Invio richiesta al LLM...
Errore 429: Provider returned error. Riprovo tra 5 secondi...
Invio richiesta al LLM...
Risposta ricevuta correttamente.
Il testo è stato salvato con successo nel file: /content/drive/My Drive/ing_dati/2212.01552_1_claims.json
2212.01552v1.json
Invio richiesta al LLM...
Risposta ricevuta correttamente.
Il testo è stato salvato con successo nel file: /content/drive/My Drive/ing_dati/2212.01552_2_claims.json
1806.06207v1.json
Invio richiesta al LLM...
Errore 429: Provider returned error. Riprovo tra 5 secondi...
Invio richiesta al LLM...
Errore 429: Provider returned error. Riprovo tra 5 secondi...
Invio richiesta al LLM...
Errore 429: Provider returned error. Riprovo tra 5 secondi...
Invio richiesta al LLM...
Errore 429: Provider returned error. Riprovo tra 5 secondi...
Invio richiesta al LLM...
Errore 429: Rate limit exceeded: limit_rpm/google/gemini-2.0-flash-thinking-exp-1219/7582deca-4e94-40ab-8ccc-d70f164857c5. Riprovo tra 5 secondi...

KeyboardInterrupt: 